In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import torch
import numpy as np
from pathlib import Path
from geqtrain.scripts.evaluate import load_model, evaluate
from csnet.training.dataset import build_input_data
from tqdm.notebook import tqdm
from geqtrain.data import AtomicDataDict

torch.set_default_dtype(torch.float32)

In [ ]:
model_path = "/home/angiod@usi.ch/CSnet/results/SHIFTX2/production-geqmodel-noise/best_model.pth"
# pdb_file_list =   [
#     '/storage_common/angiod/A2A/Vincenzo/FAN.pdb',
#     '/storage_common/angiod/A2A/Vincenzo/pAs.pdb',
#     '/storage_common/angiod/A2A/Vincenzo/INzma.pdb',
# ]
# traj_files_list = [
#     ['/storage_common/angiod/A2A/Vincenzo/FAN_strided.xtc'],
#     ['/storage_common/angiod/A2A/Vincenzo/pAs_strided.xtc'],
#     ['/storage_common/angiod/A2A/Vincenzo/INzma_strided.xtc'],
# ]
# npy_out_list = [
#     'cs_FAN_geqmodel.npy',
#     'cs_pAs_geqmodel.npy',
#     'cs_INzma_geqmodel.npy',
# ]
pdb_file_list =   [
    '/storage_common/angiod/TPP1/tpp1.gro',
    '/storage_common/angiod/TPP1/tpp1.gro',
    '/storage_common/angiod/TPP1/tpp1.gro',
]
traj_files_list = [
    ['/storage_common/angiod/TPP1/cluster1.trr'],
    ['/storage_common/angiod/TPP1/cluster2.trr'],
    ['/storage_common/angiod/TPP1/cluster3.trr'],
]
npy_out_list = [
    '../temp/cs_c1_geqmodel_noise.npy',
    '../temp/cs_c2_geqmodel_noise.npy',
    '../temp/cs_c3_geqmodel_noise.npy',
]
device = "cuda:3"

In [ ]:
model, config = load_model(Path(model_path), device=device)

for pdb_file, traj_files, npy_out in zip(pdb_file_list, traj_files_list, npy_out_list):
    batches, dataset = build_input_data(pdb_file, r_max=config.get("r_max"), traj_files=traj_files, selection="protein")
    break
    cs = []
    for batch in tqdm(batches):
        try:
            for v in batch.values():
                if isinstance(v, torch.Tensor):
                    v.to(device)
            cs.append(evaluate(
                model=model,
                batch=batch,
                node_out_keys=[AtomicDataDict.NODE_OUTPUT_KEY]
            )[AtomicDataDict.NODE_OUTPUT_KEY].cpu().numpy())
        except:
            pass
    cs = np.stack(cs, axis=0)
    np.save(npy_out, cs)

In [ ]:
from plot import plot_violin, plot_distribution, plot_violindist

atoms_filters = ['*_C_*',]

for atoms_filter in atoms_filters: 
    fltr = np.ones_like(dataset['atom_fullnames'], dtype=bool)
    res_fltr, atom_fltr, resid_fltr = atoms_filter.split('_')
    fltr = fltr * (~np.isin(dataset['atom_resnames'], ['ACE']))
    if res_fltr != '*':
        fltr = fltr * np.isin(dataset['atom_resnames'], res_fltr.split('-'))
    if atom_fltr != '*':
        fltr = fltr * np.isin(dataset['atom_names'].flatten(), atom_fltr.split('-'))
    if resid_fltr != '*':
        fltr = fltr * np.isin(dataset['atom_resnumbers'], [int(x) for x in resid_fltr.split('-')])

    atom_names = dataset['atom_fullnames'][fltr]

    ds_names = npy_out_list # ['../temp/cs_FAN.npy', '../temp/cs_pAs.npy', '../temp/cs_INzma.npy'][::-1]
    cs = []
    for ds_name in ds_names:
        x = np.load(ds_name)
        cs.append(x.reshape(len(x), -1))
    chemical_shifts = [x[:, fltr] for x in cs]

    # plot_distribution(atom_names, chemical_shifts, ds_names, 'Chemical Shifts', figsize=(20,8), overlap_same=False)
    plot_violin(atom_names, chemical_shifts, ds_names, 'Chemical Shifts', overlap_same=False)
    # plot_violindist(chemical_shifts, ds_names, 'Chemical Shifts', figsize=(20,8))

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

def show2d(ds_names, df_ref):
  x_fltr = np.nonzero(
        #                                             all   his    asn    gln    trp
            np.isin(dataset['atom_names'].flatten(), ['N', 'ND1', 'ND2', 'NE2', 'NE1']) \
          * ~np.isin(dataset['atom_resnames'], ['NME']) \
        # * np.isin(dataset['atom_resnumbers'], [225])
        )[0]
  x_coord = dataset["coords"][0, x_fltr]

  y_fltr =  np.nonzero(
    #                                         all   his    asn     asn     gln      gln    trp
    np.isin(dataset['atom_names'].flatten(), ['H', 'HD1' 'HD21', 'HD22', 'HE21', 'HE22', 'HE1'])
  )[0]
  y_coord = dataset["coords"][0, y_fltr]

  final_x_fltr = []
  final_y_fltr = []
  for _x_coord, _x_fltr in zip(x_coord, x_fltr):
    dist = np.linalg.norm(y_coord - _x_coord, axis=-1)
    nearest_id = np.argmin(dist)
    if dist[nearest_id] < 2.0:
      final_x_fltr.append(_x_fltr)
      final_y_fltr.append(y_fltr[nearest_id])
  final_x_fltr = np.array(final_x_fltr)
  final_y_fltr = np.array(final_y_fltr)

  N_atom_names = dataset['atom_names'][final_x_fltr]
  H_atom_names = dataset['atom_names'][final_y_fltr]
  atom_resnames = dataset['atom_resnames'][final_x_fltr]
  atom_resnumbers = dataset['atom_resnumbers'][final_x_fltr]

  cs = []
  for ds_name in ds_names:
      ds = np.load(ds_name)
      cs.append(ds.reshape(len(ds), -1))
  x_cs = [elem[:, final_x_fltr] for elem in cs]
  y_cs = [elem[:, final_y_fltr] for elem in cs]

  for i, (conformation_ds_x, conformation_ds_y) in enumerate(zip(x_cs, y_cs)):
    mean_ds_x = np.mean(conformation_ds_x, axis=0)
    mean_ds_y = np.mean(conformation_ds_y,axis=0)
    median_ds_x = np.median(conformation_ds_x, axis=0)
    median_ds_y = np.median(conformation_ds_y,axis=0)
    min_ds_x = mean_ds_x - np.min(conformation_ds_x, axis=0)
    min_ds_y = mean_ds_y - np.min(conformation_ds_y,axis=0)
    max_ds_x = np.max(conformation_ds_x, axis=0) - mean_ds_x
    max_ds_y = np.max(conformation_ds_y,axis=0) - mean_ds_y
    std_ds_x = conformation_ds_x.std(axis=0)
    std_ds_y = conformation_ds_y.std(axis=0)
    df = pd.DataFrame({
      "mean_ds_x": mean_ds_x,
      "mean_ds_y": mean_ds_y,
      "median_ds_x": median_ds_x,
      "median_ds_y": median_ds_y,
      "std_ds_x": std_ds_x,
      "std_ds_y": std_ds_y,
      "min_ds_x": min_ds_x,
      "min_ds_y": min_ds_y,
      "max_ds_x": max_ds_x,
      "max_ds_y": max_ds_y,
      "residue": [f"{rna}{rnu}{Nan}-{rna}{rnu}{Han}" for rna, rnu, Nan, Han in zip(atom_resnames, atom_resnumbers, N_atom_names, H_atom_names)],
    })
    fig = go.Figure(go.Scatter(
            x = mean_ds_x,
            y = mean_ds_y,
    ))

    fig = px.scatter(df, x="median_ds_x", y="median_ds_y", color="residue",
                     error_x="std_ds_x", error_y="std_ds_y",
                    #error_x="max_ds_x", error_y="max_ds_y",
                    #error_x_minus="min_ds_x", error_y_minus="min_ds_y"
                    )
    fig.add_scatter(x=df_ref['N'], y=df_ref['H'], mode='markers', marker=dict(color="Black", symbol='x'), hovertext=[str(x) for x in df_ref['residue']])

    fig.show()
    fig.write_html(f"../media/TPP1-{i}.html")

In [ ]:
residues = []
N_val = []
H_val = []
with open("../media/TPP1/tpp1.list", 'r') as f:
    for line in f.readlines():
        residue, N, H = line.strip().split()
        residues.append(residue)
        N_val.append(N)
        H_val.append(H)
residues = np.array(residues)
N_val = np.array(N_val)
H_val = np.array(H_val)
df_ref = {
    "residue": residues,
    "N": N_val,
    "H": H_val,
}

In [ ]:
show2d(npy_out_list, df_ref)